In [2]:
import pandas as pd

Read data from files, Verify the number of reviews that were read (100,000 in total):

In [3]:
train=pd.read_csv("/Users/alenadenisova/Desktop/Linguists/word2vec-nlp-tutorial/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "/Users/alenadenisova/Desktop/Linguists/word2vec-nlp-tutorial/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "/Users/alenadenisova/Desktop/Linguists/word2vec-nlp-tutorial/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



Import various modules for string cleaning

In [4]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords   

Function to convert a document to a sequence of words,optionally removing stop words. Returns a list of words:

In [5]:
def review_to_wordlist( review, remove_stopwords=False ):
    review_text = BeautifulSoup(review).get_text()     # 1. Remove HTML
    review_text = re.sub("[^a-zA-Z]"," ", review_text) # 2. Remove non-letters
    words = review_text.lower().split()                # 3. Convert words to lower case and split them
    if remove_stopwords:                               # 4. Optionally remove stop words (false by default)
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return(words)                                      # 5. Return a list of words

Download the punkt tokenizer for sentence splitting:

In [6]:
import nltk.data
#nltk.download() 

Load the punkt tokenizer:

In [7]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

Define a function to split a review into parsed sentences:

In [8]:
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    
    raw_sentences = tokenizer.tokenize(review.strip())  # 1. Use the NLTK tokenizer to split the paragraph into sentences
    sentences = []                                      # 2. Loop over each sentence
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:                        # If a sentence is empty, skip it
            sentences.append( review_to_wordlist( raw_sentence, \
            remove_stopwords ))                        # Otherwise, call review_to_wordlist to get a list of words
    
    return sentences                             # 3. Return the list of sentences(each sentence is a list of words,
                                                        # so this returns a list of lists

In [9]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


/Users/alenadenisova/anaconda3/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/alenadenisova/anaconda3/lib/python3.10/site-packages/bs4/__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


Parsing sentences from unlabeled set


In [10]:
# Check how many sentences we have in total - should be around 850,000+
print(len(sentences))

794002


In [11]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [12]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


Import the built-in logging module and configure it so that Word2Vec creates nice output messages:

In [13]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)


Set values for various parameters:

In [14]:
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

Initialize and train the model:

In [15]:
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            vector_size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling init_sims will make the model more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2024-04-15 10:45:23,652 : INFO : collecting all words and their counts
2024-04-15 10:45:23,652 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-04-15 10:45:23,685 : INFO : PROGRESS: at sentence #10000, processed 225908 words, keeping 17776 word types
2024-04-15 10:45:23,711 : INFO : PROGRESS: at sentence #20000, processed 452097 words, keeping 24953 word types
2024-04-15 10:45:23,742 : INFO : PROGRESS: at sentence #30000, processed 671984 words, keeping 30044 word types
2024-04-15 10:45:23,764 : INFO : PROGRESS: at sentence #40000, processed 898757 words, keeping 34358 word types


Training model...


2024-04-15 10:45:23,785 : INFO : PROGRESS: at sentence #50000, processed 1122100 words, keeping 37824 word types
2024-04-15 10:45:23,813 : INFO : PROGRESS: at sentence #60000, processed 1341946 words, keeping 40782 word types
2024-04-15 10:45:23,834 : INFO : PROGRESS: at sentence #70000, processed 1566792 words, keeping 43383 word types
2024-04-15 10:45:23,857 : INFO : PROGRESS: at sentence #80000, processed 1785875 words, keeping 45762 word types
2024-04-15 10:45:23,879 : INFO : PROGRESS: at sentence #90000, processed 2010133 words, keeping 48195 word types
2024-04-15 10:45:23,900 : INFO : PROGRESS: at sentence #100000, processed 2231529 words, keeping 50243 word types
2024-04-15 10:45:23,921 : INFO : PROGRESS: at sentence #110000, processed 2451527 words, keeping 52128 word types
2024-04-15 10:45:23,942 : INFO : PROGRESS: at sentence #120000, processed 2674780 words, keeping 54171 word types
2024-04-15 10:45:23,963 : INFO : PROGRESS: at sentence #130000, processed 2899962 words, keep

2024-04-15 10:45:25,488 : INFO : PROGRESS: at sentence #770000, processed 17252420 words, keeping 121803 word types
2024-04-15 10:45:25,509 : INFO : PROGRESS: at sentence #780000, processed 17481011 words, keeping 122484 word types
2024-04-15 10:45:25,545 : INFO : PROGRESS: at sentence #790000, processed 17710319 words, keeping 123190 word types
2024-04-15 10:45:25,554 : INFO : collected 123504 word types from a corpus of 17798270 raw words and 794002 sentences
2024-04-15 10:45:25,555 : INFO : Creating a fresh vocabulary
2024-04-15 10:45:25,620 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=40 retains 16490 unique words (13.35% of original 123504, drops 107014)', 'datetime': '2024-04-15T10:45:25.619966', 'gensim': '4.3.2', 'python': '3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ]', 'platform': 'macOS-12.4-arm64-arm-64bit', 'event': 'prepare_vocab'}
2024-04-15 10:45:25,620 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=40 leaves 17239125 word corpus

2024-04-15 10:46:00,029 : INFO : not storing attribute cum_table
2024-04-15 10:46:00,077 : INFO : saved 300features_40minwords_10context


Exploring the Model Results:

In [16]:
model.wv.doesnt_match("man woman child kitchen".split())

'kitchen'

In [17]:
model.wv.doesnt_match("france england germany berlin".split())

'berlin'

In [18]:
model.wv.most_similar("man")

[('woman', 0.6165322065353394),
 ('lady', 0.5824472904205322),
 ('lad', 0.562603235244751),
 ('men', 0.5167946219444275),
 ('monk', 0.5155864953994751),
 ('guy', 0.5074439644813538),
 ('soldier', 0.5072890520095825),
 ('millionaire', 0.5027772188186646),
 ('businessman', 0.5011990070343018),
 ('person', 0.4968433380126953)]

In [19]:
model.wv.most_similar("green")

[('blue', 0.6756093502044678),
 ('yellow', 0.6158223152160645),
 ('soylent', 0.5916862487792969),
 ('red', 0.5511486530303955),
 ('tinted', 0.5438907146453857),
 ('lighted', 0.5380668640136719),
 ('fur', 0.532646119594574),
 ('flashlight', 0.5303142070770264),
 ('colored', 0.521892249584198),
 ('silver', 0.5187628269195557)]

In [20]:
model.wv.most_similar("good")

[('decent', 0.6954634189605713),
 ('bad', 0.6345804929733276),
 ('great', 0.6118682622909546),
 ('nice', 0.5847127437591553),
 ('lousy', 0.5817535519599915),
 ('fine', 0.563975989818573),
 ('passable', 0.5555528998374939),
 ('mediocre', 0.55186927318573),
 ('cool', 0.5485385656356812),
 ('watchable', 0.5241504907608032)]

In [21]:
model.wv.most_similar("awful")

[('terrible', 0.7726731300354004),
 ('atrocious', 0.7428853511810303),
 ('dreadful', 0.7237646579742432),
 ('horrible', 0.7184943556785583),
 ('abysmal', 0.6965259909629822),
 ('horrendous', 0.6892979145050049),
 ('appalling', 0.6839365363121033),
 ('horrid', 0.6575130224227905),
 ('crappy', 0.610087513923645),
 ('amateurish', 0.6070759892463684)]

In [22]:
# Load the model that we created
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

2024-04-15 10:46:00,202 : INFO : loading Word2Vec object from 300features_40minwords_10context
2024-04-15 10:46:00,219 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2024-04-15 10:46:00,220 : INFO : setting ignored attribute cum_table to None
2024-04-15 10:46:00,302 : INFO : Word2Vec lifecycle event {'fname': '300features_40minwords_10context', 'datetime': '2024-04-15T10:46:00.302853', 'gensim': '4.3.2', 'python': '3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ]', 'platform': 'macOS-12.4-arm64-arm-64bit', 'event': 'loaded'}


In [23]:
type(model.wv.vectors) # vectors вместо syn0

numpy.ndarray

KeyedVectors.load_word2vec_format instead of  Word2Vec.load_word2vec_format

word2vec_model.wv.save_word2vec_format instead of  word2vec_model.save_word2vec_format

model.wv.syn0norm instead of  model.syn0norm

model.wv.syn0 instead of  model.syn0

model.wv.vocab instead of model.vocab

model.wv.index2word instead of  model.index2word

In [24]:
model.wv.vectors.shape

(16490, 300)

In [25]:
model.wv["flower"]

array([ 0.0061881 ,  0.02481586,  0.00612303,  0.07100722, -0.00914465,
        0.03688773,  0.01628157,  0.05933855,  0.0604945 , -0.07720757,
        0.02967836,  0.07661617,  0.00264418, -0.04631488, -0.05260531,
        0.00100371,  0.02289619, -0.13530129, -0.05669887,  0.01918228,
       -0.05199756,  0.00798491,  0.01625661,  0.0367662 ,  0.14335768,
       -0.00121609, -0.0162967 , -0.02419809, -0.04577519,  0.05443172,
        0.04023934, -0.02037233, -0.01740628, -0.04092016, -0.06050093,
       -0.00884334, -0.01062836, -0.05201737, -0.00326537,  0.06021222,
       -0.04865903,  0.04232333,  0.08118019,  0.0285969 , -0.11885691,
       -0.02770385,  0.11113984,  0.0771514 ,  0.01775114,  0.02233298,
        0.11325468,  0.06049047, -0.04097607, -0.02647992, -0.0054828 ,
        0.01072516,  0.0730335 , -0.0010708 , -0.03390911, -0.0086191 ,
       -0.14693284,  0.09438164,  0.08965211,  0.05836939,  0.02989194,
        0.01673451, -0.10435382, -0.02144188,  0.01198142, -0.06

In [26]:
type(model.wv["flower"])

numpy.ndarray

From Words To Paragraphs, Attempt 1: Vector Averaging

In [27]:
import numpy as np  # Make sure that numpy is imported

Function to average all of the word vectors in a given paragraph:

In [28]:
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")   # 1.Pre-initialize an empty numpy array (for speed)
    nwords = 0.
    
    index2word_set = set(model.wv.index_to_key)    # 2.Index2word is a list that contains the names of the words in 
                                                  # the model's vocabulary. Convert it to a set, for speed 
    for word in words:                        # 3.Loop over each word in the review and, if it is in the model's 
        if word in index2word_set:                #vocaublary, add its feature vector to the total
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model.wv[word])
   
    featureVec = np.divide(featureVec,nwords) # Divide the result by the number of words to get the average
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
   
    counter = 0                                                              # 1. Initialize a counter
    
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32") # 2. Preallocate a 2D numpy array, for speed
    
    for review in reviews:                                                    # 3. Loop through the reviews 
        if counter%1000 == 0:                                               # 3.1. Print a status message every 1000th review
            print("Review %d of %d" % (counter, len(reviews)))
            reviewFeatureVecs[counter] = makeFeatureVec(review, model,       # 3.2.Call the function that makes average feature vectors
           num_features)
        counter = counter + 1                                                 # 4. Increment the counter
    return reviewFeatureVecs

Calculate average feature vectors for training and testing sets, using the functions we defined above. Notice that we now use stop word
removal.

In [29]:
clean_train_reviews = []

for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")

clean_test_reviews = []

for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

/Users/alenadenisova/anaconda3/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

In [30]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )
print('ready')

Fitting a random forest to labeled training data...
ready


From Words to Paragraphs, Attempt 2: Clustering ("vector quantization")

In [31]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

In [32]:
word_vectors = model.wv.vectors # instead of model.syn0
num_clusters = int(word_vectors.shape[0] / 5)

#Setting "k" (num_clusters) to be 1/5th of the vocabulary size, or an average of 5 words per cluster

In [33]:
type(num_clusters)

int

In [34]:
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Initalize a k-means object and use it to extract centroids

/Users/alenadenisova/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [35]:
# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  485.24791288375854 seconds.


In [36]:
# Create a Word / Index dictionary, mapping each vocabulary word to a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index_to_key, idx ))

In [37]:
# For the first 10 clusters
for cluster in range(0,10):
    print("\nCluster %d" % cluster)        # Print the cluster number  
    words = []                            # Find all of the words for that cluster number, and print them out        
    for i in range(0,len(list(word_centroid_map.values()))):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print(words)
    
    #the_values = dict.values()
#SUM = sum(list(the_values)[1:10])


Cluster 0
['kiddies', 'spectators', 'westerners']

Cluster 1
['posse']

Cluster 2
['janeane']

Cluster 3
['spirit', 'wrath', 'backbone', 'grapes']

Cluster 4
['scheming', 'stubborn', 'meek', 'reckless', 'conniving', 'treacherous', 'shrewd', 'calculating', 'sullen', 'headstrong', 'penniless']

Cluster 5
['prop', 'chained', 'burying', 'whipping', 'chatting', 'clinging', 'locking']

Cluster 6
['baker', 'francis', 'alexander', 'antonio', 'cole', 'freddie', 'bryan', 'houston', 'banderas', 'jacob', 'rudolph', 'chang', 'rene', 'caruso', 'rudolf', 'giovanni', 'klaus', 'luigi', 'whitman']

Cluster 7
['organized', 'awaiting', 'condemned', 'sacrificed', 'immune']

Cluster 8
['filled', 'abound', 'laced', 'interspersed', 'riddled', 'sprinkled', 'rife', 'peppered', 'littered']

Cluster 9
['release', 'public', 'industry', 'market', 'marketing', 'publicity', 'distribution', 'funding', 'promotion', 'distributor', 'distributors']


In python-3.X dict.values doesn't return a list object like how it used to perform in python-2.X. 
In python-3.x it returns a dict-value object which is a set-like object and uses a hash table for storing its items which is not suitable for indexing. This feature, in addition to supporting most of set attributes, is very optimized for operations like membership checking (using in operator).
By assigning dict.values() to a list you are not converting it to a list; you are just storing it as dict_values (may be an object). To convert it write value_list=list(dict.values()). Now even while printing the value_list you will get the list elements and not dict_values(......).



now we have a cluster (or "centroid") assignment for each word, and we can define a function to convert reviews into bags-of-centroids. This works just like Bag of Words but uses semantically related clusters instead of individual words:

In [38]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    num_centroids = max( word_centroid_map.values() ) + 1   #The number of clusters is equal to the highest cluster
                                                            #index in the word/centroid map

    bag_of_centroids = np.zeros( num_centroids, dtype="float32" ) # Pre-allocate the bag of centroids vector (for speed)
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [39]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, word_centroid_map )
    counter += 1

In [40]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [41]:
#.syn0 --> .wv.vectors
#.index2word --> .wv.index_to_key
#dictionary[i] --> list(dictionary.values())[i]

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

In [43]:
def print_results(results):
    print('BEST PARAMS: {}]'.format(results.best_params_))
    
    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+-{}) for {}'.format(round(mean,3), round(std * 2, 3), params))

In [44]:
rf = RandomForestClassifier()
parameters = {
    'n_estimators': [100, 250],
    'max_depth': [16]
}
cv = GridSearchCV(rf, parameters, cv = 3)
cv.fit(train_centroids,train["sentiment"])

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [16], 'n_estimators': [100, 250]})

In [45]:
print_results(cv)

BEST PARAMS: {'max_depth': 16, 'n_estimators': 250}]
0.836 (+-0.008) for {'max_depth': 16, 'n_estimators': 100}
0.842 (+-0.007) for {'max_depth': 16, 'n_estimators': 250}


In [46]:
cv.best_estimator_

RandomForestClassifier(max_depth=16, n_estimators=250)